In [1]:
import numpy as np
import glob
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import pandas as pd
import random

print("Hello World")
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

Hello World
GPU name:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Now we wish to load in the data. For now this will just be in the local directory

data_dir = "D:\\POLYGON_TRAINING\\"
file_list = glob.glob(data_dir + "*.npy" )
random.shuffle(file_list)
file_list = file_list[:1800]
print(str(len(file_list)) + " files found")
batch_size = 200


1800 files found


In [3]:
#Ok let's try and do a simple model

model = Sequential()

model.add ( Conv2D(64, (3, 3), input_shape = (128,128,16), padding = "same")  )
model.add (Activation("relu"))
model.add (MaxPooling2D(pool_size = (2,2)))
model.add( BatchNormalization() )

model.add ( Conv2D(64, (3, 3) , padding = "same") )
model.add (Activation("relu"))
model.add (MaxPooling2D(pool_size = (2,2)))
model.add( BatchNormalization() )

model.add ( Conv2D(128, (3, 3) , padding = "same") )
model.add (Activation("relu"))
model.add (MaxPooling2D(pool_size = (2,2)))
model.add( BatchNormalization() )


model.add ( Conv2D(128, (3, 3) , padding = "same") )
model.add (Activation("relu"))
model.add (MaxPooling2D(pool_size = (2,2)))
model.add( BatchNormalization() )


model.add ( Conv2D(128, (3, 3) , padding = "same") )
model.add (Activation("relu"))
model.add (MaxPooling2D(pool_size = (2,2)))
model.add( BatchNormalization() )


model.add ( Conv2D(256, (3, 3) , padding = "same") )
model.add ( Dropout (0.4))
model.add (Activation("relu"))
model.add (MaxPooling2D(pool_size = (2,2)))
model.add( BatchNormalization() )

model.add (UpSampling2D(2, interpolation = 'bilinear'))
model.add (Activation("relu"))
model.add( BatchNormalization() )


model.add (UpSampling2D(2, interpolation = 'bilinear'))
model.add (Activation("relu"))
model.add( BatchNormalization() )

model.add (UpSampling2D(2, interpolation = 'bilinear'))
model.add (Activation("relu"))
model.add( BatchNormalization() )

model.add (UpSampling2D(2, interpolation = 'bilinear'))
model.add (Activation("relu"))
model.add( BatchNormalization() )


model.add (UpSampling2D(2, interpolation = 'bilinear'))
model.add (Activation("relu"))
model.add( BatchNormalization() )


model.add (UpSampling2D(2, interpolation = 'bilinear'))
model.add (Activation("relu"))
model.add( BatchNormalization() )


model.add ( Conv2D (2,(3,3),activation="sigmoid",padding="same"))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      9280      
_________________________________________________________________
activation (Activation)      (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0

In [4]:
#Ok let's see what happens if we try and train the model
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

model.compile(optimizer = 'adam',loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
# we can't fit huge amounts of data simultaneously so we need to split into batches

sat_data_list = np.zeros((batch_size,128,128,16))
poly_data_list = np.zeros((batch_size,128,128,2))

for j in range(int(len(file_list)/batch_size)):
    print("processing batch " + str(j+1) + " of " + str(len(file_list)/batch_size))
    for idx, file in enumerate(file_list[batch_size*j:batch_size*(j+1)]):
        file_data = np.load(file)

        # Seperate out the actual data from the desired output and replace NaNs with 0
        sat_data = np.nan_to_num(file_data[192:320,192:320,:16])
        poly_data = np.dstack((np.nan_to_num(file_data[192:320,192:320,16]),1-(np.nan_to_num(file_data[192:320,192:320,16]))))

        sat_data_normalised = tf.keras.utils.normalize(sat_data,axis = -1)
        poly_data_normalised = tf.keras.utils.normalize(poly_data)
        # This will ideally need normalising before going any further. poly_data should already be 0 or 1

        #The Neural network requires an extra dimension which is the concatenation of all the images
        sat_data_list[idx] = sat_data_normalised[:,:,:]
        poly_data_list[idx] = poly_data_normalised[:,:,:]
    # Ok now we have the data. Let's rename them X and Y for brevity

    X = sat_data_list
    Y = poly_data_list

    model.fit(X,Y, epochs = 12, batch_size = 45, validation_split = 0.1,verbose = 1)


processing batch 1 of 9.0
Epoch 1/12
2/2 [==============================] - 29s 925ms/step - loss: 0.8622 - accuracy: 0.5230 - val_loss: 0.6809 - val_accuracy: 0.9398
Epoch 2/12
2/2 [==============================] - 0s 221ms/step - loss: 0.7650 - accuracy: 0.5770 - val_loss: 0.6797 - val_accuracy: 0.9398
Epoch 3/12
2/2 [==============================] - 0s 215ms/step - loss: 0.7161 - accuracy: 0.5901 - val_loss: 0.6782 - val_accuracy: 0.9398
Epoch 4/12
2/2 [==============================] - 0s 218ms/step - loss: 0.6827 - accuracy: 0.6360 - val_loss: 0.6737 - val_accuracy: 0.9398

In [9]:
#Ok let's see what this predicts

test_dir = "D:\\POLYGON_VALIDATION\\"
file_list_test = glob.glob(test_dir + "*.npy" )
print(str(len(file_list_test)) + " files found for testing")

# Now we use numpy.load to open the files

sat_data_list_test = np.zeros((len(file_list_test),128,128,16))
poly_data_list_test = np.zeros((len(file_list_test),128,128,2))

for idx, file in enumerate(file_list_test):
    file_data = np.load(file)

    # Seperate out the actual data from the desired output and replace NaNs with 0
    sat_data_test = np.nan_to_num(file_data[192:320,192:320,:16])
    poly_data_test = np.dstack((np.nan_to_num(file_data[192:320,192:320,16]),1-(np.nan_to_num(file_data[192:320,192:320,16]))))

    sat_data_normalised_test = tf.keras.utils.normalize(sat_data_test,axis = -1)
    poly_data_normalised_test = tf.keras.utils.normalize(poly_data_test)
    # This will ideally need normalising before going any further. poly_data should already be 0 or 1

    #The Neural network requires an extra dimension which is the concatenation of all the images
    sat_data_list_test[idx] = sat_data_normalised_test
    poly_data_list_test[idx] = poly_data_normalised_test
# Ok now we have the data. Let's rename them X and Y for brevity

X_test = sat_data_list_test
Y_test = poly_data_list_test

print("About the make a prediction. Jeronimo!! ")

prediction = model.predict(X_test)

print("Prediction generated. ")
np.ndarray.view(prediction)
for i in range(0,6):
    plt.imsave("prediction_" + str(i) + ".png",prediction[i,:,:,0] , cmap = "binary")
    plt.imsave("true_" + str(i) + ".png",Y_test[i,:,:,0] , cmap = "binary")
    plt.imsave("IR_" + str(i) + ".png",X_test[i,:,:,12] , cmap = "PuOr")

7 files found for testing
About the make a prediction. Jeronimo!! 
Prediction generated. 
